In [9]:
import json
import os
import chi
import chi.ssh
import chi.network
from datetime import datetime, timedelta
from dateutil import tz
import chi.server

In [2]:
import chi,os

project_id = "CHI-231095"
site_name = "KVM@TACC"
# tell python-chi what project to use, and where
chi.set("project_name", project_id)  
chi.use_site(site_name)
# also set environment variables, for benefit of future commands
os.environ["OS_PROJECT_NAME"] = project_id
os.environ["OS_REGION_NAME"] = site_name

Now using KVM@TACC:
URL: https://kvm.tacc.chameleoncloud.org
Location: Austin, Texas, USA
Support contact: help@chameleoncloud.org


In [3]:
username = os.getenv("USER")
prefix = username + "_k8s-ml_"
server_name = prefix + 'Server'
network_name = prefix + 'Net'
subnet_name = prefix + 'Subnet'
router_name = prefix + 'Router'
lease_name = prefix + 'Lease'

In [4]:
#Server attributes
image_name='CC-Ubuntu18.04'
node_type="compute_cascadelake_r"
server_count=3
physical_network="physnet1"
subnet_cidr = '192.168.1.0/24'

In [5]:
os_conn = chi.clients.connection()

In [6]:
network = os_conn.create_network(
                network_name,
            )
network_id = network['id']

print(f'Network ID: {network_id}')


Network ID: 2d95f71c-d3bc-4b9f-a23d-b7935c53f40e


In [7]:
subnet = chi.network.create_subnet(subnet_name, network_id, cidr=subnet_cidr, gateway_ip=None)
router = chi.network.create_router(router_name, gw_network_name='public')
chi.network.add_subnet_to_router(router['id'], subnet['id'])

{'network_id': '2d95f71c-d3bc-4b9f-a23d-b7935c53f40e',
 'tenant_id': '287158fd03da48e48eb0d2682f38fcc9',
 'subnet_id': 'dae13aa0-f2c6-4a16-8ef0-f4563c03dedb',
 'subnet_ids': ['dae13aa0-f2c6-4a16-8ef0-f4563c03dedb'],
 'port_id': '3e36625b-3f87-41ae-8faa-15ff9e95908b',
 'id': '35999376-bb00-48f1-97ae-b029686051eb'}

In [11]:
servers = []
for i in range(server_count):
    servers.append(chi.server.create_server(server_name+f"-{i}", 
                                  network_id=network_id,
                                  nics=[{"net-id": network_id,"v4-fixed-ip": f"192.168.1.1{i}" }],
                                  image_name=image_name,
                                            flavor_name= "m1.medium",
                                  count=1))

In [12]:
nodes = {}

for j,i in enumerate(servers):
    chi.server.wait_for_active(i.id)
    floating_ip = chi.server.associate_floating_ip(i.id)
    nodes[j] = floating_ip

    print(f'Floating IP of node_{j}: {floating_ip}')

Floating IP of node_0: 129.114.26.25
Floating IP of node_1: 129.114.27.86
Floating IP of node_2: 129.114.26.233


In [16]:
for j,i in enumerate(servers):
    nova_server = chi.nova().servers.get(i.id)
    nova_server.add_security_group("Allow SSH")
    print(f"updated security groups: {[group.name for group in nova_server.list_security_group()]}")

updated security groups: ['Allow SSH', 'default']
updated security groups: ['Allow SSH', 'default']
updated security groups: ['Allow SSH', 'default']


In [17]:
for node_ip in nodes.values():
    chi.server.wait_for_tcp(node_ip, port=22)

In [18]:
from chi.ssh import Remote
for node_ip in nodes.values():
    remote = Remote(node_ip)
    nova=chi.clients.nova()
    [kp.name for kp in nova.keypairs.list()]
    keypair_name = 'id_rsa_chameleon' # The name of the key they uploaded 
    public_key = nova.keypairs.get(keypair_name).public_key # Contents of the public key
    remote.run(f"echo {public_key} >> ~/.ssh/authorized_keys")

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:835: UserWarning: Unknown ssh-ed25519 host key for 129.114.26.25: b'faa2dc1aa4420c08e115533e02233127'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/paramiko/client.py:835: UserWarning: Unknown ssh-ed25519 host key for 129.114.27.86: b'b2874bb8f45f5d95b43e909c4e2ca74a'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/paramiko/client.py:835: UserWarning: Unknown ssh-ed25519 host key for 129.114.26.233: b'bd456547899e267037add87814b54e84'
  warnings.warn(


In [19]:
for node_ip in nodes.values():
    remote = chi.ssh.Remote(node_ip)
    remote.run('ssh-keygen -t rsa -b 4096 -f ~/.ssh/id_rsa -q -N ""')
    public_key = remote.run('cat ~/.ssh/id_rsa.pub').tail("stdout")[2:]
    public_key = public_key.replace("\n", "")
    for other_node_ip in nodes.values():
        remote = chi.ssh.Remote(other_node_ip)
        remote.run(f'echo {public_key} >> ~/.ssh/authorized_keys') 

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDReQTLF+4WZ1DHz2PFftgEwWSnA+JtkjaQfW1lyLxXMX90v3xJ6Tf7+X1NDFFTQlcDQsm7GKIgEF54OhMWT/pQ12pmUG/1KYRT4jSIHaRn1Z/3ASfUkLt5glbVKtCXoup44ZtvtpvYiFVvFUZcl9TQVx3w2TimFwj25j6rSvrKsMSqrxigESYwFkG8yYMeN55aYBNgocFWgq1eIgq1IV2JeJv9Po5L2vK1sTGJZNZktKTU3xi7ZpvAsOwKzDLo8SgOfvTnlOgr4skimHMdJ9D8/VCmqCVWKjNyYBHspWk5aSuZVcLyKtVuZME0uHtEZA4fTuSt08ntf8C1ae4u2h1F+2wFx5kvKzF+cqhjkLGZRC9cl+D3e1IvyOZPEYkiLtRzEdN6wv2dYEBncrZyWcw5AMFQ4NeicdGwCtnMI3fotVjjdYSbooNrEL29W7P8BxmJGHEw+YCYB8ebv3LtvSgEHnRcTnWjtmPNiHdxEeHDd3wbx7u/Bi/DUqSTYuER4weyhmCT8U7LEOoEJmf392CZ8WMQ5SqfYvb7Q/3NGslcJ0ZCWPsDPfl62w8/vnNMJfPmboQNMF7dKDRtX15eydGhWDKtvJF9VU704EfN38gSxbEWRl0xcMPldi+BzWsaVzOG+BeM7GxBFztLYLepzkJUEJ+a3Qm1jbJEunKWjIaWqQ== cc@cp3793-nyu-edu-k8s-ml-server-0
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDjHzWfxnLEmA6SPrvGRVvx1NPu9Z33dPH1BX1X4X8OGK5cRJgsIdirfxK+cD3fNofhL3nzNP2y5xBJHnf78Z7TTSHCdOZSdtF3du3Uz8wSSy03ipL7wfVSwlHJLhXurwakUDQNASW6Yr6gxNVXm57+uHP5Lzsv952JMiouxAn6gtYSm4Af3aNXiIgE1jX7QWKE6tmeoHM0Q

In [20]:
physical_ips=[]
for node_ip in nodes.values():
    chi.server.wait_for_tcp(node_ip, port=22)
    remote = chi.ssh.Remote(node_ip)
    physical_ips.append(remote.run('hostname -I').tail("stdout")[2:-1])

192.168.1.10 
192.168.1.11 
192.168.1.12 


In [21]:
ansible_node = nodes[0]
remote = chi.ssh.Remote(ansible_node)
remote.run("sudo apt install virtualenv")
remote.run("virtualenv -p python3 myenv")
remote.run("git clone https://github.com/kubernetes-sigs/kubespray.git")
remote.run("cd kubespray; git checkout release-2.16")
remote.run("source myenv/bin/activate; cd kubespray; pip3 install -r requirements.txt")
remote.run("cd kubespray; cp -rfp inventory/sample inventory/mycluster")
remote.run("rm -rf kubespray/contrib/inventory_builder/inventory.py")
remote.run("rm -rf kubespray/inventory/mycluster/group_vars/k8s_cluster/addons.yml")
remote.run("wget https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/inventory.py -O kubespray/contrib/inventory_builder/inventory.py")
remote.run("wget https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/addons.yml -O kubespray/inventory/mycluster/group_vars/k8s_cluster/addons.yml")
remote.run(f"source myenv/bin/activate; declare -a IPS=({physical_ips[0]} {physical_ips[1]} {physical_ips[2]});"+"cd kubespray; CONFIG_FILE=inventory/mycluster/hosts.yaml python3 contrib/inventory_builder/inventory.py ${IPS[@]}")

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  virtualenv
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 4476 B of archives.
After this operation, 30.7 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu bionic/universe amd64 virtualenv all 15.1.0+ds-1.1 [4476 B]


debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 4476 B in 0s (25.9 kB/s)
Selecting previously unselected package virtualenv.
(Reading database ... 63329 files and directories currently installed.)
Preparing to unpack .../virtualenv_15.1.0+ds-1.1_all.deb ...
Unpacking virtualenv (15.1.0+ds-1.1) ...
Setting up virtualenv (15.1.0+ds-1.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
created virtual environment CPython3.6.9.final.0-64 in 1631ms
  creator CPython3Posix(dest=/home/cc/myenv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/home/cc/.local/share/virtualenv)
    added seed packages: pip==21.3.1, setuptools==59.6.0, wheel==0.37.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


Cloning into 'kubespray'...


Branch 'release-2.16' set up to track remote branch 'release-2.16' from 'origin'.


Switched to a new branch 'release-2.16'


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for ansible: filename=ansible-2.9.20-py3-none-any.whl size=16203449 sha256=0cfe220088b4a90de9f03a147d4d109ac108cd071c7b2ac4200e21b9b346ab73
  Stored in directory: /home/cc/.cache/pip/wheels/0f/36/b7/907fa25c2228a59030698d2ac557ba5c694b648dede82fb7e1
Successfully built ansible


--2023-05-18 21:18:48--  https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/inventory.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17940 (18K) [text/plain]
Saving to: ‘kubespray/contrib/inventory_builder/inventory.py’

     0K .......... .......                                    100% 26.9M=0.001s

2023-05-18 21:18:48 (26.9 MB/s) - ‘kubespray/contrib/inventory_builder/inventory.py’ saved [17940/17940]

--2023-05-18 21:18:48--  https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/addons.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected

DEBUG: Adding group all
DEBUG: Adding group kube_control_plane
DEBUG: Adding group kube_node
DEBUG: Adding group etcd
DEBUG: Adding group k8s_cluster
DEBUG: Adding group calico_rr
DEBUG: adding host node-0 to group all
DEBUG: adding host node-1 to group all
DEBUG: adding host node-2 to group all
DEBUG: adding host node-0 to group etcd
DEBUG: adding host node-1 to group etcd
DEBUG: adding host node-2 to group etcd
DEBUG: adding host node-0 to group kube_control_plane
DEBUG: adding host node-1 to group kube_control_plane
DEBUG: adding host node-0 to group kube_node
DEBUG: adding host node-1 to group kube_node
DEBUG: adding host node-2 to group kube_node


<Result cmd='source myenv/bin/activate; declare -a IPS=(192.168.1.10 192.168.1.11 192.168.1.12);cd kubespray; CONFIG_FILE=inventory/mycluster/hosts.yaml python3 contrib/inventory_builder/inventory.py ${IPS[@]}' exited=0>

In [22]:
remote.run("source myenv/bin/activate; sudo rm -rf /usr/local/lib/python3.6/dist-packages/OpenSSL")
remote.run("source myenv/bin/activate; sudo rm -rf /usr/local/lib/python3.6/dist-packages/pyOpenSSL-22.1.0.dist-info/")
remote.run("source myenv/bin/activate; sudo pip3 install pyOpenSSL==22.0.0")

<Result cmd='source myenv/bin/activate; sudo pip3 install pyOpenSSL==22.0.0' exited=0>

In [23]:
remote.run("source myenv/bin/activate; cd kubespray; ansible-playbook -i inventory/mycluster/hosts.yaml  --become --become-user=root cluster.yml")


PLAY [localhost] ***************************************************************
Thursday 18 May 2023  21:19:37 +0000 (0:00:00.088)       0:00:00.088 ********** 

TASK [Check 2.9.0 <= Ansible version < 2.11.0] *********************************
ok: [localhost] => {
    "changed": false,
    "msg": "All assertions passed"
}
Thursday 18 May 2023  21:19:38 +0000 (0:00:00.035)       0:00:00.124 ********** 

TASK [Check that python netaddr is installed] **********************************
ok: [localhost] => {
    "changed": false,
    "msg": "All assertions passed"
}
Thursday 18 May 2023  21:19:38 +0000 (0:00:00.018)       0:00:00.142 ********** 

TASK [Check that jinja is not too old (install via pip)] ***********************
ok: [localhost] => {
    "changed": false,
    "msg": "All assertions passed"
}


[WARNING]: Could not match supplied host pattern, ignoring: kube-master



PLAY [Add kube-master nodes to kube_control_plane] *****************************
skipping: no hosts matched


[WARNING]: Could not match supplied host pattern, ignoring: kube-node



PLAY [Add kube-node nodes to kube_node] ****************************************
skipping: no hosts matched

PLAY [Add k8s-cluster nodes to k8s_cluster] ************************************
skipping: no hosts matched


...

[WARNING]: flush_handlers task does not support when conditional


Thursday 18 May 2023  21:20:42 +0000 (0:00:00.071)       0:01:04.941 ********** 
Thursday 18 May 2023  21:20:42 +0000 (0:00:00.069)       0:01:05.010 ********** 

TASK [container-engine/containerd-common : gather os specific variables] *******
ok: [node-1] => (item=/home/cc/kubespray/roles/container-engine/containerd-common/vars/../vars/ubuntu.yml)
ok: [node-0] => (item=/home/cc/kubespray/roles/container-engine/containerd-common/vars/../vars/ubuntu.yml)
ok: [node-2] => (item=/home/cc/kubespray/roles/container-engine/containerd-common/vars/../vars/ubuntu.yml)
Thursday 18 May 2023  21:20:43 +0000 (0:00:00.141)       0:01:05.152 ********** 
Thursday 18 May 2023  21:20:43 +0000 (0:00:00.091)       0:01:05.243 ********** 
Thursday 18 May 2023  21:20:43 +0000 (0:00:00.067)       0:01:05.311 ********** 
Thursday 18 May 2023  21:20:43 +0000 (0:00:00.062)       0:01:05.373 ********** 
Thursday 18 May 2023  21:20:43 +0000 (0:00:00.070)       0:01:05.444 ********** 
Thursday 18 May 2023  21:20:43

[WARNING]: noop task does not support when conditional



TASK [download : prep_kubeadm_images | Create kubeadm config] ******************
changed: [node-0]
changed: [node-1]
Thursday 18 May 2023  21:21:42 +0000 (0:00:00.652)       0:02:05.047 ********** 

TASK [prep_kubeadm_images | Copy kubeadm binary from download dir to system path] ***
changed: [node-1]
changed: [node-0]
Thursday 18 May 2023  21:21:43 +0000 (0:00:00.578)       0:02:05.625 ********** 

TASK [download : prep_kubeadm_images | Set kubeadm binary permissions] *********
ok: [node-0]
ok: [node-1]
Thursday 18 May 2023  21:21:43 +0000 (0:00:00.339)       0:02:05.965 ********** 

TASK [download : prep_kubeadm_images | Generate list of required images] *******
ok: [node-0]
Thursday 18 May 2023  21:21:44 +0000 (0:00:00.448)       0:02:06.414 ********** 

TASK [download : prep_kubeadm_images | Parse list of images] *******************
ok: [node-0] => (item=k8s.gcr.io/kube-apiserver:v1.20.7)
ok: [node-0] => (item=k8s.gcr.io/kube-controller-manager:v1.20.7)
ok: [node-0] => (item=k8s.g

<Result cmd='source myenv/bin/activate; cd kubespray; ansible-playbook -i inventory/mycluster/hosts.yaml  --become --become-user=root cluster.yml' exited=0>

In [29]:
remote = chi.ssh.Remote(nodes[0])
remote.run("sudo cp -R /root/.kube /home/cc/.kube; sudo chown -R cc /home/cc/.kube; sudo chgrp -R cc /home/cc/.kube")

<Result cmd='sudo cp -R /root/.kube /home/cc/.kube; sudo chown -R cc /home/cc/.kube; sudo chgrp -R cc /home/cc/.kube' exited=0>

In [25]:
for i in nodes.keys():
    remote = chi.ssh.Remote(nodes[i])
    remote.run("sudo ufw allow 32000")

Rule added
Rule added (v6)
Rule added
Rule added (v6)
Rule added
Rule added (v6)


In [26]:
remote = chi.ssh.Remote(nodes[0])
remote.run("sudo groupadd docker; sudo usermod -aG docker $USER")
remote = chi.ssh.Remote(nodes[0])

groupadd: group 'docker' already exists


In [27]:
remote.run("docker run -d -p 5000:5000 --restart always --name registry registry:2")
remote.run("sudo wget https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/daemon.json -O /etc/docker/daemon.json")
remote.run("sudo service docker restart")

Unable to find image 'registry:2' locally
2: Pulling from library/registry
8a49fdb3b6a5: Pulling fs layer
58116d8bf569: Pulling fs layer
4cb4a93be51c: Pulling fs layer
cbdeff65a266: Pulling fs layer
6b102b34ed3d: Pulling fs layer
cbdeff65a266: Waiting
6b102b34ed3d: Waiting
58116d8bf569: Verifying Checksum
58116d8bf569: Download complete
4cb4a93be51c: Verifying Checksum
4cb4a93be51c: Download complete
8a49fdb3b6a5: Verifying Checksum
8a49fdb3b6a5: Download complete
cbdeff65a266: Verifying Checksum
cbdeff65a266: Download complete
6b102b34ed3d: Verifying Checksum
6b102b34ed3d: Download complete
8a49fdb3b6a5: Pull complete
58116d8bf569: Pull complete
4cb4a93be51c: Pull complete
cbdeff65a266: Pull complete
6b102b34ed3d: Pull complete
Digest: sha256:20d084723c951e377e1a2a5b3df316173a845e300d57ccdd8ae3ab2da3439746
Status: Downloaded newer image for registry:2


4dc02ea4416e8fdc3702f05fdb044056c8e9f39216b7746a2e96b55471c7c459


--2023-05-18 21:31:50--  https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/daemon.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48 [text/plain]
Saving to: ‘/etc/docker/daemon.json’

     0K                                                       100% 1.16M=0s

2023-05-18 21:31:51 (1.16 MB/s) - ‘/etc/docker/daemon.json’ saved [48/48]



<Result cmd='sudo service docker restart' exited=0>

In [30]:
remote.run("kubectl get nodes")

NAME     STATUS   ROLES                  AGE     VERSION
node-0   Ready    control-plane,master   6m7s    v1.20.7
node-1   Ready    control-plane,master   5m38s   v1.20.7
node-2   Ready    <none>                 4m43s   v1.20.7


<Result cmd='kubectl get nodes' exited=0>

In [31]:
remote.run("docker run hello-world")

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world
719385e32844: Pulling fs layer
719385e32844: Download complete
719385e32844: Pull complete
Digest: sha256:fc6cf906cbfa013e80938cdf0bb199fbdbb86d6e3e013783e5a766f50f5dbce0
Status: Downloaded newer image for hello-world:latest



Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



<Result cmd='docker run hello-world' exited=0>

In [32]:
nodes[0]

'129.114.26.25'